In [17]:
import os

from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone

## 환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

In [18]:
## 문서 읽고 분할 #####################################################
import secret
## 문서 파일 목록
pdf_paths = [ 
             '스페셜리포트_2025년10대미래기술.pdf'
             ]

## 문서 로드(읽어오기)
documents = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

documents

[Document(metadata={'producer': 'iLovePDF', 'creator': 'Adobe InDesign 20.0 (Windows)', 'creationdate': '2025-01-10T11:36:54+09:00', 'trapped': '/False', 'moddate': '2025-01-10T05:23:57+00:00', 'source': '스페셜리포트_2025년10대미래기술.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'Adobe InDesign 20.0 (Windows)', 'creationdate': '2025-01-10T11:36:54+09:00', 'trapped': '/False', 'moddate': '2025-01-10T05:23:57+00:00', 'source': '스페셜리포트_2025년10대미래기술.pdf', 'total_pages': 24, 'page': 1, 'page_label': '2'}, page_content='소형언어모델\n베라 루빈 천문대\n장기지속형 HIV 예방제\n생성형 AI 검색\n소 트림 감소제\n청정 제트연료\n고속학습 로봇\n효과적인 줄기세포 치료\n로보택시\n녹색철강\n10 Breakthrough Technologies 2025\nMIT 테크놀로지 리뷰가 선정한 2025년 10대 미래 기술\n MIT 테크놀로지 리뷰가 2025년 10대 미래 기술을 선정했습니다. 소형언어모델부터 베라 루빈 천문대와 \n로보텍시 및 고속학습 로봇과 녹색철강 등에 이르기까지 전 세계에 실질적인 영향을 줄 것으로 기대되는 \n유망한 기술들을 소개합니다.\n 125년 전통의 세계적인 IT·테크 미디어 MIT 테크놀로지 리뷰가 2025년 가장 주목할 10대 미래 기술\n(Breakthrough Technologies)을 발표했다.'),

In [19]:
## 청크 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

## 문서 분할
document_list = text_splitter.split_documents(documents)

len(document_list)

31

In [20]:

## 임베딩 -> 벡터 스토어(데이터베이스)에 저장
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'mit10th'

# 파인콘: 저장
database =  PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name,
)

## 저장된 인덱스 가져오기
## [방법 1]
# database = PineconeVectorStore(
#     index=pc.Index(index_name),
#     embedding=embedding,
# )

## [방법 2]
# database = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding,
# )

